In [1]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [423 kB]
Hit:12 http://ppa.launchpad.net

In [ ]:
!sudo apt-get install cdo

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra fonts-droid-fallback fonts-lyx
  fonts-noto-mono ghostscript gsfonts javascript-common libblosc1 libcdi0
  libcupsfilters1 libcupsimage2 libdxflib3 libeccodes-data libeccodes0
  libemos-data libemos0d libgs9 libgs9-common libijs-0.35 libjbig2dec0
  libjs-jquery libjs-jquery-ui libmagics++-data libmagplus3v5 libodb-api-0d
  libterralib3 libudunits2-0 libudunits2-data poppler-data
  python-matplotlib-data python-tables-data python3-bottleneck python3-bs4
  python3-cdo python3-chardet python3-cycler python3-dask python3-dateutil
  python3-decorator python3-html5lib python3-lxml python3-matplotlib
  python3-netcdf4 python3-numexpr python3-numpy python3-olefile python3

In [ ]:
!pip install netCDF4
!pip install nctoolkit
!pip install cdo
!pip install udunitspy
!pip install cfunits
!pip install cf-python
from google.colab import drive
drive.mount('/content/gdrive')
import netCDF4 as nc
import numpy as np
import seaborn as sns
import pandas as pd
from shapely.geometry import Polygon
import shapely
import matplotlib.pyplot as plt
import os
from datetime import datetime
from tqdm.auto import tqdm
import xarray as xr
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib
import nctoolkit as nct
import cf

import keras
from keras.models import Sequential, load_model
from keras.layers import Activation, Dense, Dropout, Input, BatchNormalization
from keras.objectives import MSE, MAE
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import gc
import sys
from cdo import *
cdo = Cdo()

     |████████████████████████████████| 102kB 2.5MB/s 
     |████████████████████████████████| 3.1MB 11.3MB/s 
     |████████████████████████████████| 15.8MB 273kB/s 
     |████████████████████████████████| 512kB 42.1MB/s 
     |████████████████████████████████| 337kB 37.5MB/s 
     |████████████████████████████████| 12.1MB 38.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.5MB 43.9MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 215kB 49.9MB/s 
     |████████████████████████████████| 225kB 41.5MB/s 
     |████████████████████████████████| 706kB 35.0MB/s 
     |████████████████████████████████| 112kB 54.6MB/s 
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.whl size=12516146 sha256=850539b36c505e2da447406bb2e999d47aa26a1c2e3d6f3d3f9897e2e4406a1f
  Stored in directory: /root/.c

**Define global variables**

In [ ]:
lat_bnds, lon_bnds = [-10.7, 3.51], [28.5, 40.7]

In [ ]:
date_range = pd.date_range('2016-01-01', '2020-12-31')
date_range = date_range.strftime('%Y%m%d').to_list()

**TAMSAT**

In [ ]:
one = xr.open_dataset('/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/TAMSAT/bounding_box_daily/01-tamsatDaily.v3.1-1262304000-1622502000_28.417_41.194_-11.183_3.622.nc')
one = one.where(one.apply(np.isfinite)).fillna(np.nan)

In [ ]:
one.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/TAMSAT/dataex.nc')

In [ ]:
data = nct.open_data('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/TAMSAT/dataex.nc')

In [ ]:
data.to_latlon(lon = lon_bnds, lat = lat_bnds, res = [0.05, 0.05])
data = data.to_xarray()
data.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/TAMSAT_regridded')

**IMERG**

In [ ]:
list_datasets = []
for date in tqdm(date_range):
  dataset = nct.open_data('/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/IMERG/3B-DAY-L.MS.MRG.3IMERG.' + date + '-S000000-E235959.V06.nc4')
  dataset.to_latlon(lon = lon_bnds, lat = lat_bnds, res = [0.05, 0.05])
  dataset = dataset.to_xarray()
  list_datasets.append(dataset)
dataset = xr.concat(list_datasets, dim = 'time')
dataset.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/IMERG_regridded')

**CMORPH**

**CHIRPS**

In [ ]:

dataset = nct.open_data('/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CHIRPS/daily/05/chirps-v2.0.2020.days_p05.nc')
dataset.to_latlon(lon = lon_bnds, lat = lat_bnds, res = [0.05, 0.05])
dataset = dataset.to_xarray()
dataset.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/CHIRPS_regridded_2020')

In [ ]:
dataset

<xarray.Dataset>
Dimensions:  (lat: 285, lon: 245, time: 366)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-12-31
  * lon      (lon) float32 28.5 28.55 28.6 28.65 28.7 ... 40.55 40.6 40.65 40.7
  * lat      (lat) float32 -10.7 -10.65 -10.6 -10.55 -10.5 ... 3.35 3.4 3.45 3.5
Data variables:
    precip   (time, lat, lon) float32 ...
Attributes: (12/17)
    CDI:               Climate Data Interface version ?? (http://mpimet.mpg.d...
    history:           Fri May 21 14:54:04 2021: cdo -L -remap,/tmp/nctoolkit...
    institution:       Climate Hazards Group.  University of California at Sa...
    Conventions:       CF-1.6
    title:             CHIRPS Version 2.0
    version:           Version 2.0
    ...                ...
    comments:           time variable denotes the first day of the given day....
    acknowledgements:  The Climate Hazards Group InfraRed Precipitation with ...
    ftp_url:           ftp://chg-ftpout.geog.ucsb.edu/pub/org/chg/products/CH...
    website:           http://chg.geog.ucsb.edu/data/chirps/index.html
    faq:               http://chg-wiki.geog.ucsb.edu/wiki/CHIRPS_FAQ
    CDO:               Climate Data Operators version 1.9.3 (http://mpimet.mp...

In [ ]:
years = ['2016', '2017', '2018', '2019', '2020']

list_datasets = []
for year in (years):
  dataset = nct.open_data('/content/gdrive/MyDrive/rainfall-forecast/precipitation_data/CHIRPS/daily/05/chirps-v2.0.' + year + '.days_p05.nc')
  dataset.to_latlon(lon = lon_bnds, lat = lat_bnds, res = [0.05, 0.05])
  dataset = dataset.to_xarray()
  list_datasets.append(dataset)
dataset = xr.concat(list_datasets, dim = 'time')a
dataset.to_netcdf('/content/gdrive/MyDrive/rainfall-forecast/merge_model/ConvLSTM/predictors/predictors_regridded/CHIRPS_regridded')

In [ ]:
two = 